In [ ]:
import requests


webhook_url = 'https://discord.com/api/webhooks/1313346849838596106/6Rn_8BNDeL9bMYfFtqscpu4hPah5c2RsNl0rBiPoSw_Qb9RXgDdVHoHmwEuStPv_ufnV'  # 복사한 Discord 웹훅 URL로 변경
username = "Stock Trading Bot"

message = "매수가 완료되었습니다."

data = {
    "content": message,
    "username": username,  # 원하는 이름으로 설정 가능
}

# 요청 보내기
response = requests.post(webhook_url, json=data)

# 응답 확인
if response.status_code == 204:
    print("메시지가 성공적으로 전송되었습니다.")
else:
    print(f"메시지 전송 실패: {response.status_code}, {response.text}")

In [ ]:
import os

# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = "/Users/hsh/dev_project/sb-fsts"
os.chdir(project_root)  # 루트 경로로 변경

In [ ]:
from app.utils.crud_sql import SQLExecutor
from app.utils.database import get_db, get_db_session

sql_executor = SQLExecutor()

In [14]:
query = """
        SELECT 
            *
        FROM 
            fsts.test_table
    """

params = {}

with get_db_session() as db:
    result = sql_executor.execute_select(db, query, params)

print(result)

[{'id': 'id1', 'name': 'name1'}, {'id': 'id2', 'name': 'name2'}]


In [13]:
# RETURNING * 꼭 붙여야 함.
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

id = 'id2'
name = 'name2'

params = {
    "id": id,
    "name": name
}

with get_db_session() as db:
    result = sql_executor.execute_insert(db, query, params)

Insert succeeded: [{'id': 'id2', 'name': 'name2'}]


In [14]:
from pykis.kis import PyKis, KisAccessToken

kis_id = 'dreaminmind'  # 한국투자증권 HTS ID
app_key = "PSyTGF07QupJyV76XGm3mkgcr4RDvSeODpVZ"
secret_key = "eteoHNN+iHktbHC1TOKNdDc2ecFHqwyA+o1OijESqRtWY2cirhUqbiuFfO5zmEPNqB8/P0RSBuTjZnPq4zc5u3dKHIg/HOFQqmZcCik621aWqti5MBReqNpr/NChcs8edoBKd4cgJaC47m3IKncU4GglKzWNqHtic/4X8lmOAZx0oDGuFkI="
account = "67737279"

kis = PyKis(
    id=kis_id,             # 한국투자증권 HTS ID
    appkey=app_key,    # 발급받은 App Key
    secretkey=secret_key, # 발급받은 App Secret
    account=account, # 계좌번호 (예: "12345678-01")
    # keep_token=True           # 토큰 자동 갱신 여부
)

In [16]:
from pykis import KisBalance


account = kis.account()
balance: KisBalance = account.balance()

for holding in balance.stocks:
    print(holding)
    symbol = holding.symbol
    original_symbol_name = holding.symbol_name

KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='036540', qty=161, price=3140, amount=505540, profit=6440, profit_rate=1.290322580645161290322580645)


AttributeError: 'KisDomesticBalanceStock' object has no attribute 'symbol_name'

In [12]:
import boto3
import json

s3_client = boto3.client('s3', region_name='ap-northeast-2', endpoint_url='https://s3.ap-northeast-2.amazonaws.com', config=boto3.session.Config(signature_version='s3v4'))
bucket_name="sb-fsts"

token_save_path = f"credentials/pykis/token.json"

response = s3_client.get_object(Bucket=bucket_name, Key=token_save_path)

# 본문 읽기 및 JSON 파싱
content = response['Body'].read().decode('utf-8')
data = json.loads(content)
data
with open("token.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [13]:
kis = PyKis(
    id=kis_id,             # 한국투자증권 HTS ID
    appkey=app_key,    # 발급받은 App Key
    secretkey=secret_key, # 발급받은 App Secret
    account=account, # 계좌번호 (예: "12345678-01")
    token=KisAccessToken.load("token.json"),
    keep_token=True           # 토큰 자동 갱신 여부
)
kis.token.save("token.json")

KisHTTPError: HTTP 요청에 실패했습니다.
(403) Forbidden
{"error_description":"접근토큰 발급 잠시 후 다시 시도하세요(1분당 1회)","error_code":"EGW00133"}

[  Request  ]: POST https://openapi.koreainvestment.com:9443/oauth2/tokenP
Headers: {'User-Agent': 'PyKis/2.1.3', 'Accept-Encoding': 'gzip, deflate, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '283', 'Content-Type': 'application/json'}
Params: {}
Body: [PROTECTED BODY]

In [50]:
s3_client.upload_file(
    Filename="token.json",
    Bucket=bucket_name,
    Key=token_save_path
)